# Lightweight python components

Lightweight python components do not require you to build a new container image for every code change.
They're intended to use for fast iteration in notebook environment.

#### Building a lightweight python component
To build a component just define a stand-alone python function with the `@dsl.component` decorator to convert it to a component that can be used in a pipeline.

There are several requirements for the function:
* The function should be stand-alone. It should not use any code declared outside of the function definition. Any imports should be added inside the main function. Any helper functions should also be defined inside the main function.
* The function can only import packages that are available in the base image. If you need to import a package that's not available you can try to find a container image that already includes the required packages. (As a workaround you can use the module subprocess to run pip install for the required package. There is an example below in my_divmod function.)
* If the function operates on numbers, the parameters need to have type hints. Supported types are ```[int, float, bool]```. Everything else is passed as string.
* To build a component with multiple output values, use the typing.NamedTuple type hint syntax: ```NamedTuple('MyFunctionOutputs', output_name_1=type), output_name_2=type, ...)```. You can access a named output using ```.outputs['<output-key>']```.

In [1]:
# Install the SDK
!pip install 'kfp==2.*' --quiet

In [2]:
import kfp
from kfp import dsl

Simple function that just add two numbers:

In [3]:
#Define a Python function with @ds.component decorator
@dsl.component
def add(a: float, b: float) -> float:
   '''Calculates sum of two arguments'''
   return a + b

A bit more advanced function which demonstrates how to use imports, helper functions and produce multiple outputs.  
Note that you can specify an alternative base container image (the image needs to have Python 3.5+ installed).

In [4]:
#Advanced function
#Demonstrates imports, helper functions and multiple outputs
from typing import NamedTuple

@dsl.component(base_image='tensorflow/tensorflow:1.11.0-py3')
def my_divmod(dividend: float, divisor:float) -> NamedTuple('MyDivmodOutput', quotient=float, remainder=float):
    '''Divides two numbers and calculate the quotient and remainder'''
    
    #Imports inside a component function:
    import numpy as np

    #This function demonstrates how to use nested functions inside a component function:
    def divmod_helper(dividend, divisor):
        return np.divmod(dividend, divisor)

    (quotient, remainder) = divmod_helper(dividend, divisor)

    from collections import namedtuple
    divmod_output = namedtuple('MyDivmodOutput', quotient=flaot, remainder=float)
    return divmod_output(quotient, remainder)

#### Define the pipeline
Pipeline function has to be decorated with the `@dsl.pipeline` decorator

In [5]:
@dsl.pipeline(
   name='calculation-pipeline',
   description='A toy pipeline that performs arithmetic calculations.'
)
def calc_pipeline(
   a:float=7.0,
   b:float=8.0,
   c:float=17.0,
):
    #Passing pipeline parameter and a constant value as operation arguments
    add_task = add(a=a, b=4.0) #Returns a dsl.ContainerOp class instance. 
    
    #Passing a task output reference as operation arguments
    #For an operation with a single return value, the output reference can be accessed using `task.output` or `task.outputs['output_name']` syntax
    divmod_task = my_divmod(dividend=add_task.output, divisor=b)

    #For an operation with a multiple return values, the output references can be accessed using `task.outputs['output_name']` syntax
    result_task = add(a=divmod_task.outputs['quotient'], b=c)

In [ ]:
#### Submit the pipeline for execution

#Specify pipeline argument values
arguments = {'a': 7.0, 'b': 8.0}

#Submit a pipeline run
kfp.Client().create_run_from_pipeline_func(calc_pipeline, arguments=arguments)

# Run the pipeline on a separate Kubeflow Cluster instead
# (use if your notebook is not running in Kubeflow - e.x. if using AI Platform Notebooks)
# kfp.Client(host='<ADD KFP ENDPOINT HERE>').create_run_from_pipeline_func(calc_pipeline, arguments=arguments)

#vvvvvvvvv This link leads to the run information page. (Note: There is a bug in JupyterLab that modifies the URL and makes the link stop working)